In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import pickle
from tensorflow.keras.models import Sequential 
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import csv

In [2]:
"""
Isear : Constants
This file will incorporate all needed constants in the first time
"""

"""
Here are the ISEAR codes explained

ID : identifier of the subject
CITY : city in which lives the subject

COUN : country of the subject
1 "SWEDEN" 
2 "NORWAY" 
3 "F.R.G." 
4 "FINLAND" 
5 "GREECE" 
6 "HONG KONG" 
7 "LEBANON" 
8 "AUSTRIA" 
9 "AUSTRALIA" 
10 "BRAZIL" 
11 "BOTSWANA" 
12 "BULGARIA" 
13 "FRANCE" 
14 "ITALY" 
15 "JAPAN" 
16 "NEW ZEALAND" 
17 "NETHERLANDS" 
18 "PORTUGAL" 
19 "SPAIN" 
20 "ZAMBIA" 
21 "ZIMBABWE" 
22 "USA" 
23 "POLAND" 
24 "NIGERIA" 
25 "ISRAEL" 
26 "INDIA" 
27 "MALAWI" 
28 "SWITZERLAND" 
29 "CHILE" 
30 "CHINA MAINLAND" 
31 "YUGOSLAVIA" 
32 "COSTA RICA" 
33 "HONDURAS" 
34 "MEXICO" 
35 GUATEMALA" 
36 "VENEZUELA" 
37 "EL SALVADOR"

SUBJ :

SEX : the sex of the subject
1 MALE
2 FEMALE

AGE : the age of the subject

RELI : the religion of the subject
1 "PROTESTANT" 
2 "CATHOLIC" 
3 "JEWISH" 
4 "HINDU" 
5 "BUDDHIST" 
6 "NATIVE" 
7 "OTHERS" 
8 "ARELIGIOUS"

PRAC : is subject practising religion
1 TRUE
2 FALSE

FOCC & MOCC : Father and Mother occupation
1 "HOUSEWIFE" 2 "UNEMPLOYED" 3 "STUDENT" 4 "BLUE COLLAR UNTRAINED" 5 "BLUE COLLAR TRAINED" 6 "WHITE COLLAR NONACADEMIC" 7 "WHITE COLLAR ACADEMIC" 8 "SELF-EMPLOYED NONACADEMIC" 9 "SELF-EMPLOYED ACADEMIC"

FIEL : subject's field of study
1 "PSYCHOLOGY" 2 "SOCIAL SCIENCES" 3 "LANGUAGES" 4 "FINE ARTS" 5 "LAW" 6 "NATURAL SCIENCE" 7 "ENGINEERING" 8 "MEDICAL" 9 "OTHER"

EMOT : subject's main emotion
1 JOY
2 FEAR
3 ANGER 
4 SADNESS
5 DISGUST
6 SHAME
7 GUILT

SIT : free text description (core of training)

-------------------- 
Event Quantification
--------------------
WHEN : when did that happen
1 DAYS AGO 2 WEEKS AGO 3 MONTHS AGO 4 YEARS AGO

LONG : how long did it last
1 MINUTES 2 HOUR 3 HOURS 4 DAY OR MORE

INTS : intensity score from 1 to 4

--------------------
Reaction
--------------------
ERGO : Ergotropic arousal (breathing, heart beat, etc) from 0 to 4

TROPHO : Trophophic arousal (lump, stomach, crying...) from 0 to 3

TEMPER : Felt temperature (-1 to 2)

--------------------
Expressive behaviour
--------------------
MOVE : from -1 (withdrawing) to +1 (forward)

EXPRES : nonverbal activity, from 0 to 6

PARAL : speech melody and tempo change from 0 to 3

EXP1 : if laughing/smilig 1, else 0
EXP2 : if sobbing/Crying
EXP10 : if moving against

CON : subject efforts to control
1 NONE 2 A LITTLE 3 VERY MUCH 0 NA

EXPC : situation expected to occur
0 NA 1 NOT 2 A LITTLE 3 VERY MUCH

PLEA : situation is pleasant

FAIR : fairness of the situation

MORL : morality of the situation

SELF : consequence on self estime

RELA : consequence on relationship with persons involved
1 NEGATIVE 2 NONE 3 POSITIVE 0 NA

PLAN : situation hindered current plans
0 HELPED 2 NO MATTER 3 HINDERED 0 NA

CAUS : who caused the situation
O NA 1 SELF 2 CLOSE 3 OTHER 4 IMPERSONNAL

COPING : ability to cope
1 NO ACTION NECESSARY
2 MANAGEABLE
3 ESCAPABLE SITUATION
4 DENIAL
5 DOMINATED
 
"""

# ID : identifier of the subject
# CITY : city in which lives the subject

# COUN : country of the subject
COUN = ["SWEDEN", "NORWAY", "F.R.G.", "FINLAND",
        "GREECE", "HONG KONG", "LEBANON", "AUSTRIA",
        "AUSTRALIA", "BRAZIL", "BOTSWANA", "BULGARIA",
        "FRANCE", "ITALY", "JAPAN", "NEW ZEALAND",
        "NETHERLANDS", "PORTUGAL", "SPAIN",
        "ZAMBIA", "ZIMBABWE", "USA", "POLAND", "NIGERIA",
        "ISRAEL", "INDIA", "MALAWI", "SWITZERLAND", "CHILE",
        "CHINA MAINLAND", "YUGOSLAVIA", "COSTA RICA", "HONDURAS",
        "MEXICO", "GUATEMALA", "VENEZUELA", "EL SALVADOR"]

# SUBJ : Identifier of the subject

# SEX : the sex of the subject
SEX = ["MALE", "FEMALE"]

# AGE : the age of the subject

# RELI : the religion of the subject
RELI = ["PROTESTANT", "CATHOLIC", "JEWISH", "HINDU",
        "BUDDHIST", "NATIVE", "OTHERS", "ARELIGIOUS"]

# PRAC : is subject practising religion
PRAC = ["TRUE", "FALSE"]

# FOCC & MOCC : Father and Mother occupation
POCC = ["HOUSEWIFE", "UNEMPLOYED", "STUDENT",
        "BLUE COLLAR UNTRAINED", "BLUE COLLAR TRAINED",
        "WHITE COLLAR NONACADEMIC", "WHITE COLLAR ACADEMIC",
        "SELF-EMPLOYED NONACADEMIC", "SELF-EMPLOYED ACADEMIC"]

# FIEL : subject's field of study
FIEL = ["PSYCHOLOGY", "SOCIAL SCIENCES", "LANGUAGES", "FINE ARTS",
        "LAW", "NATURAL SCIENCE", "ENGINEERING", "MEDICAL", "OTHER"]

# EMOT : subject's main emotion
EMOT = ["JOY", "FEAR", "ANGER", "SADNESS", "DISGUST", "SHAME", "GUILT"]

# SIT : free text description (core of training)

# --------------------
# Event Quantification
# --------------------
# WHEN : when did that happen
WHEN = ["DAYS AGO", "WEEKS AGO", "MONTHS AGO", "YEARS AGO"]

# LONG : how long did it last
LONG = ["MINUTES", "HOUR", "HOURS", "DAY OR MORE"]

# INTS : intensity score from 1 to 4

# --------------------
# Reaction
# --------------------
# ERGO : Ergotropic arousal (breathing, heart beat, etc) from 0 to 4

# TROPHO : Trophophic arousal (lump, stomach, crying...) from 0 to 3

# TEMPER : Felt temperature (-1 to 2)

# --------------------
# Expressive behaviour
# --------------------
# MOVE : from -1 (withdrawing) to +1 (forward)

# EXPRES : nonverbal activity, from 0 to 6

# PARAL : speech melody and tempo change from 0 to 3

# EXP1 : if laughing/smilig 1, else 0
# EXP2 : if sobbing/Crying
# EXP10 : if moving against

# CON : subject efforts to control
CON = ["NA", "NONE", "A LITTLE", "VERY MUCH"]

# EXPC : situation expected to occur
EXPC = CON
# PLEA : situation is pleasant
PLEA = CON
# FAIR : fairness of the situation
FAIR = CON
# MORL : morality of the situation
MORL = CON
# SELF : consequence on self estime
SELF = CON

# RELA : consequence on relationship with persons involved
RELA = ["NA", "NEGATIVE", "NONE", "POSITIVE"]

# PLAN : situation hindered current plans
PLAN = ["NA", "HELPED", "NO_MATTER", "HINDERED"]

# CAUS : who caused the situation
CAUS = ["NA", "SELF", "CLOSE", "OTHER", "IMPERSONNAL"]

# COPING : ability to cope
COPING = ["NO ACTION NECESSARY", "MANAGEABLE",
          "ESCAPABLE SITUATION", "DENIAL", "DOMINATED"]

# ---------------------------------------
# LIST OF AVAILABLE FIELDS/ATTRIBUTES
# ---------------------------------------
CONST_ISEAR_CODES = ["ID", "CITY", "COUN", "SUBJ", "SEX",
                     "AGE", "RELI", "PRAC", "FOCC", "MOCC",
                     "FIEL", "EMOT", "WHEN", "LONG", "INTS",
                     "ERGO", "TROPHO", "TEMPER", "EXPRES",
                     "MOVE", "EXP1", "EXP2", "EXP10", "PARAL",
                     "CON", "EXPC", "PLEA", "PLAN", "FAIR",
                     "CAUS", "COPING", "MORL", "SELF", "RELA",
                     "VERBAL", "NEUTRO", "Field1", "Field3",
                     "Field2", "MYKEY", "SIT", "STATE"]


In [3]:
class IsearSubset:

    def __init__(self,
                 labels,
                 values):
        self.labels = labels
        self.values = values


class IsearDataSet:

    def __init__(self,
                 data=IsearSubset([], []),
                 target=IsearSubset([], []),
                 text_data=[]):
        self.__data = data
        self.__target = target
        self.__text_data = text_data

    def get_data(self):
        return self.__data.values

    def get_target(self):
        return self.__target.values

    def get_data_label_at(self, i):
        return self.__data.labels[i]

    def get_target_label_at(self, i):
        return self.__target.labels[i]

    def get_freetext_content(self):
        return self.__text_data


class NoSuchFieldException:

    def __init__(self, field_name):
        self.message = "No such field in dataset : " + field_name

    def get_message(self):
        return self.message


class IsearLoader:

    def load_isear(self, s_isear_path):
        f_isear = open(s_isear_path, "r")
        '''
        The isear file extracted for the purpose of this initial
        loading is a pipe delimited csv-like file with headings
        '''

        isear_reader = csv.reader(f_isear,
                                  delimiter="|",
                                  quotechar='"')
        i = 0
        entry_attributes = []
        text_data = []
        entry_target = []
        for isear_row in isear_reader:
            if i == 0:
                i = i + 1
                continue
            result = self.__parse_entry(isear_row,
                                        i,
                                        text_data)
            entry_attributes.append(result["attributes"])
            entry_target.append(result["target"])
            i = i + 1
        attributes_subset = IsearSubset(self.attribute_list,
                                        entry_attributes)
        target_subset = IsearSubset(self.target_list,
                                    entry_target)
        return IsearDataSet(attributes_subset,
                            target_subset,
                            text_data)

    def __parse_entry(self,
                      isear_row,  # The row of the entry
                      index,  # row number
                      text_data):  # the text data
        i_col = 0
        l_attributes = []
        l_target = []
        # start parsing the columns
        for isear_col in isear_row:
            # we need to know to which field we are refering
            # handling the excess columns
            if i_col >= len(CONST_ISEAR_CODES):
                break

            s_cur_col = CONST_ISEAR_CODES[i_col]

            # for further test this will tell whether we are in the SIT column,
            # which is a text column
            b_is_sit = bool(s_cur_col == "SIT")
            if b_is_sit:
                if self.provide_text:
                    # should be clear enough
                    text_data.append(isear_col)
            else:
                # should be an int

                if s_cur_col in self.attribute_list:
                    i_isear_col = int(isear_col)
                    l_attributes.append(i_isear_col)

                if s_cur_col in self.target_list:
                    i_isear_col = int(isear_col)
                    l_target.append(i_isear_col)
            # next column
            i_col = i_col + 1
        # we will return a pretty "free form" object
        return {"attributes": l_attributes,
                "target": l_target}

    def __init__(self,
                 attribute_list=[],
                 target_list=[],
                 provide_text=True):
        # list of attributes to extract, please refer to enums.py
        self.attribute_list = []
        self.set_attribute_list(attribute_list)
        # list of targets to extract
        self.target_list = []
        self.set_target_list(target_list)
        # provide the text, true by default
        self.provide_text = provide_text

    # compares attribute existence in the Isear labels
    def __check_attr_exists(self, attribute):
        return attribute in CONST_ISEAR_CODES

    def set_attribute_list(self, attrs):
        """Set a list of attributes to extract

        Args:
        attrs (list):  a list of strings refering Isear fields .

        Returns:
        self. in order to ease fluent programming (loader.set().set())
        Raises:
        NoSuchFieldException

        """
        self.attribute_list = []
        for attr in attrs:
            self.add_attribute(attr)
        return self

    def set_target_list(self, target):
        """Set a list of fields to extract as target
        Args:
        attrs (list):  a list of strings refering Isear fields .

        Returns:
        self. in order to ease fluent programming (loader.set().set())
        Raises:
        NoSuchFieldException

        """
        self.target_list = []
        for tgt in target:
            self.add_target(tgt)
        return self

    def set_provide_text(self, is_provide_text):
        """ Tell the extractor whether to load the free text field.
        Behaviour is true by default

        Args:
        is_provide_text (bool): whether to provide the text field or not
        Return
        self. For fluent API
        """
        self.provide_text = is_provide_text
        return self

    def add_attribute(self, attr):
        b_att_ex = self.__check_attr_exists(attr)
        if b_att_ex is not True:
            ex = NoSuchFieldException(attr)
            raise ex
        self.attribute_list.append(attr)
        return self

    def add_target(self, attr):
        b_att_ex = self.__check_attr_exists(attr)
        if b_att_ex is not True:
            ex = NoSuchFieldException(attr)
            raise ex
        self.target_list.append(attr)
        return self

    # def load_isear(self):


In [4]:
attributes = ['EMOT','SIT']
target = ['TROPHO','TEMPER']
loader = IsearLoader(attributes, target, True)
data = loader.load_isear('isear.csv')
data.get_data() # returns attributes
data.get_target() # returns target
data.get_freetext_content() # returns the text content of the database

['During the period of falling in love, each time that we met and Ã¡ especially when we had not met for a long time.',
 'When I was involved in a traffic accident.',
 'When I was driving home after  several days of hard work, there Ã¡ was a motorist ahead of me who was driving at 50 km/hour and Ã¡ refused, despite his low speeed to let me overtake.',
 'When I lost the person who meant the most to me. ',
 "The time I knocked a deer down - the sight of the animal's Ã¡ injuries and helplessness.  The realization that the animal was Ã¡ so badly hurt that it had to be put down, and when the animal Ã¡ screamed at the moment of death.",
 'When I did not speak the truth.',
 'When I caused problems for somebody because he could not keep the Ã¡ appointed time and this led to various consequences.',
 'When I got a letter offering me the Summer job that I had applied Ã¡ for.  ',
 'When I was going home alone one night in Paris and a man came up Ã¡ behind me and asked me if I was not afraid to be o

In [5]:
d = {'Emotion':  data.get_data(),
        'Text': data.get_freetext_content()}

df = pd.DataFrame (d, columns = ['Emotion','Text'])

In [6]:
df['Output']=0
df['Label']=0
for i in range(len(df)):
  temp=df.iloc[i,0]
  #print(max(temp))
  df.at[i,'Output']=max(temp)

df=df.drop(columns="Emotion")
df.head()

,Text,Output,Label
0,"During the period of falling in love, each tim...",1,0
1,When I was involved in a traffic accident.,2,0
2,When I was driving home after several days of...,3,0
3,When I lost the person who meant the most to me.,4,0
4,The time I knocked a deer down - the sight of ...,5,0


In [7]:
X=df['Text']
y=[]

for i in range(len(df)):
  l=[0,0]
  temp=df.iloc[i,1]
  if temp>1:
    temp=0 #Negative emotion
  else:
    temp=1 #Positive emotion
  l[temp-1]=1
  y.append(l)

y

[[1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 

In [8]:
"Number of characters in each text"

max_len=0
min_len=1000
sum_len=0
avg_len=0


for i in X:
    if len(i)>max_len:
        max_len=len(i)
    if len(i)< min_len:
        min_len=len(i)
    sum_len=sum_len+len(i)
avg_len=sum_len/len(X)
print(max_len)
print(min_len)
print(sum_len)
print(avg_len)
print(len(X))

903
5
880400
114.84476911035742
7666


In [9]:
"Number of words in each text"

max_len=0
min_len=1000
sum_len=0
avg_len=0


for i in X:
    if len(i.split())>max_len:
        max_len=len(i.split())
    if len(i.split())< min_len:
        min_len=len(i.split())
    sum_len=sum_len+len(i.split())
avg_len=sum_len/len(X)
print(max_len)
print(min_len)
print(sum_len)
print(avg_len)
print(len(X))

191
1
175008
22.829115575267416
7666


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [11]:
train_x = []
test_x = []
#Splitting pd.Series to list
data_to_list = X_train.values.tolist()
for i in range(len(data_to_list)):
    train_x.append(data_to_list[i])

data2_to_list = X_test.values.tolist()
for i in range(len(data2_to_list)):
    test_x.append(data2_to_list[i])

In [12]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

sequences = tokenizer.texts_to_sequences(train_x)
X_train = pad_sequences(sequences, maxlen=max_len)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(test_x)

sequences = tokenizer.texts_to_sequences(test_x)
X_test = pad_sequences(sequences, maxlen=max_len)

In [13]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.LSTM(20,dropout=0.6))
model.add(layers.Dense(2,activation='softmax'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 40)           200000    
                                                                 
 lstm_1 (LSTM)               (None, 20)                4880      
                                                                 
 dense_1 (Dense)             (None, 2)                 42        
                                                                 
Total params: 204,922
Trainable params: 204,922
Non-trainable params: 0
_________________________________________________________________


In [19]:
checkpoint1 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history=model.fit(X_train, y_train, epochs=40,validation_data=(X_test, y_test), callbacks=[checkpoint1])

Epoch 1/40
192/192 [==============================] - ETA: 0s - loss: 0.4271 - accuracy: 0.8549
Epoch 00001: val_accuracy improved from -inf to 0.85854, saving model to best_model2.hdf5
192/192 [==============================] - 14s 67ms/step - loss: 0.4271 - accuracy: 0.8549 - val_loss: 0.4312 - val_accuracy: 0.8585
Epoch 2/40
192/192 [==============================] - ETA: 0s - loss: 0.3414 - accuracy: 0.8580
Epoch 00002: val_accuracy did not improve from 0.85854
192/192 [==============================] - 12s 65ms/step - loss: 0.3414 - accuracy: 0.8580 - val_loss: 0.4527 - val_accuracy: 0.8585
Epoch 3/40
192/192 [==============================] - ETA: 0s - loss: 0.2877 - accuracy: 0.8788 ETA: 2s - l
Epoch 00003: val_accuracy did not improve from 0.85854
192/192 [==============================] - 12s 65ms/step - loss: 0.2877 - accuracy: 0.8788 - val_loss: 0.4855 - val_accuracy: 0.8559
Epoch 4/40
192/192 [==============================] - ETA: 0s - loss: 0.2609 - accuracy: 0.9015
Epoch

Epoch 29/40
192/192 [==============================] - ETA: 0s - loss: 0.1330 - accuracy: 0.9514 ETA: 0s - loss: 0.1
Epoch 00029: val_accuracy did not improve from 0.85854
192/192 [==============================] - 14s 73ms/step - loss: 0.1330 - accuracy: 0.9514 - val_loss: 0.7203 - val_accuracy: 0.8370
Epoch 30/40
192/192 [==============================] - ETA: 0s - loss: 0.1328 - accuracy: 0.9511
Epoch 00030: val_accuracy did not improve from 0.85854
192/192 [==============================] - 15s 76ms/step - loss: 0.1328 - accuracy: 0.9511 - val_loss: 0.7354 - val_accuracy: 0.8299
Epoch 31/40
192/192 [==============================] - ETA: 0s - loss: 0.1292 - accuracy: 0.9525
Epoch 00031: val_accuracy did not improve from 0.85854
192/192 [==============================] - 13s 70ms/step - loss: 0.1292 - accuracy: 0.9525 - val_loss: 0.7421 - val_accuracy: 0.8253
Epoch 32/40
192/192 [==============================] - ETA: 0s - loss: 0.1268 - accuracy: 0.9530
Epoch 00032: val_accuracy di

Python 3.7.6


In [20]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(2,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])


In [21]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 40)           200000    
                                                                 
 bidirectional (Bidirectiona  (None, 40)               9760      
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 2)                 82        
                                                                 
Total params: 209,842
Trainable params: 209,842
Non-trainable params: 0
_________________________________________________________________


In [22]:
checkpoint1 = ModelCheckpoint("best_model3.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history=model2.fit(X_train, y_train, epochs=40,validation_data=(X_test, y_test), callbacks=[checkpoint1])

Epoch 1/40
192/192 [==============================] - ETA: 0s - loss: 0.4200 - accuracy: 0.8531
Epoch 00001: val_accuracy improved from -inf to 0.85854, saving model to best_model3.hdf5
192/192 [==============================] - 23s 103ms/step - loss: 0.4200 - accuracy: 0.8531 - val_loss: 0.4077 - val_accuracy: 0.8585
Epoch 2/40
192/192 [==============================] - ETA: 0s - loss: 0.3483 - accuracy: 0.8571
Epoch 00002: val_accuracy did not improve from 0.85854
192/192 [==============================] - 19s 100ms/step - loss: 0.3483 - accuracy: 0.8571 - val_loss: 0.4406 - val_accuracy: 0.8585
Epoch 3/40
192/192 [==============================] - ETA: 0s - loss: 0.3062 - accuracy: 0.8795
Epoch 00003: val_accuracy did not improve from 0.85854
192/192 [==============================] - 19s 101ms/step - loss: 0.3062 - accuracy: 0.8795 - val_loss: 0.4485 - val_accuracy: 0.8559
Epoch 4/40
192/192 [==============================] - ETA: 0s - loss: 0.2604 - accuracy: 0.9033
Epoch 00004: v

192/192 [==============================] - ETA: 0s - loss: 0.1360 - accuracy: 0.9514
Epoch 00029: val_accuracy did not improve from 0.85854
192/192 [==============================] - 21s 109ms/step - loss: 0.1360 - accuracy: 0.9514 - val_loss: 0.6773 - val_accuracy: 0.7973
Epoch 30/40
192/192 [==============================] - ETA: 0s - loss: 0.1350 - accuracy: 0.9478
Epoch 00030: val_accuracy did not improve from 0.85854
192/192 [==============================] - 20s 105ms/step - loss: 0.1350 - accuracy: 0.9478 - val_loss: 0.6972 - val_accuracy: 0.8286
Epoch 31/40
192/192 [==============================] - ETA: 0s - loss: 0.1285 - accuracy: 0.9499
Epoch 00031: val_accuracy did not improve from 0.85854
192/192 [==============================] - 19s 100ms/step - loss: 0.1285 - accuracy: 0.9499 - val_loss: 0.7133 - val_accuracy: 0.7979
Epoch 32/40
192/192 [==============================] - ETA: 0s - loss: 0.1319 - accuracy: 0.9542
Epoch 00032: val_accuracy did not improve from 0.85854
19